<a href="https://colab.research.google.com/github/dvircohen0/Machine-Learning-Algorithms-From-Scratch/blob/main/VGG16_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint,EarlyStopping ,TensorBoard
import os
import tensorflow as tf
import datetime
!pip install split-folders
import splitfolders 
from keras.models import load_model
from distutils.dir_util import copy_tree
from google.colab import files 
import glob
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# download the cats and dogs dataset from Kaggle
!kaggle datasets download -d birajsth/cats-and-dogs-filtered
# unzip the dataset file
!unzip /content/cats-and-dogs-filtered.zip

In [ ]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [ ]:
# combine the train and validation folders in order to create test, train, validation folders
fromDirectory = "/content/cats_and_dogs_filtered/validation"
toDirectory = "/content/cats_and_dogs_filtered/train"
copy_tree(fromDirectory, toDirectory)
#split the combined folder into test, train, validation folders
splitfolders.ratio("/content/cats_and_dogs_filtered/train", output="Dataset", seed=1337, ratio=(.7, .15, .15), group_prefix=None)

Copying files: 3000 files [00:00, 8106.76 files/s]


In [ ]:
# create log directory in for using tensorboard
log_dir='logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

# build the train generator to load and augment the images
train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 fill_mode="nearest",
                                 preprocessing_function=preprocess_input)
train_generator=train_datagen.flow_from_directory(r"/content/Dataset/train",
                                                  target_size=(150,150),color_mode="rgb",
                                                  batch_size=16,class_mode="categorical")

# build the validation generator 
val_datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input)
val_generator=val_datagen.flow_from_directory(r"/content/Dataset/val",
                                                  target_size=(150,150),color_mode="rgb",
                                                  batch_size=16,class_mode="categorical")

Found 2100 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


In [ ]:
# using transfer learning on the VGG16 network with imagenet weights
base_model=VGG16(weights="imagenet",include_top=False,input_shape=(150,150,3))
#calculate the step per epoch size
step_size_train=train_generator.n//train_generator.batch_size
print("step size train:",step_size_train)
step_size_val=val_generator.n//val_generator.batch_size
print("step size test:",step_size_val)

VGG16_self=Sequential()
VGG16_self.add(base_model)
VGG16_self.add(Flatten())
VGG16_self.add(Dense(64,activation="relu"))
VGG16_self.add(Dense(2,activation="softmax"))
# only the added layers going to train 
base_model.trainable=False
VGG16_self.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
# callbacks for saving the best model, early stopping and tensorboard
callbacks = [
ModelCheckpoint(str(datetime.datetime.now())+"_vgg16.h5",
                monitor='val_accuracy', verbose=1, save_best_only=True,
                save_weights_only=False, mode='auto', save_freq='epoch'),
EarlyStopping(monitor='val_accuracy', min_delta=0, patience=3, verbose=1, mode='auto'),
tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)]

58892288/58889256 [==============================] - 1s 0us/step
step size train: 131
step size test: 28


In [ ]:
# train the model
VGG16_self.fit(train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=val_generator,
                    validation_steps=step_size_val,
                     callbacks=callbacks)

Epoch 1/10
131/131 [==============================] - 26s 139ms/step - loss: 0.8028 - accuracy: 0.6567 - val_loss: 0.2750 - val_accuracy: 0.9040

Epoch 00001: val_accuracy improved from -inf to 0.90402, saving model to 2021-02-24 14:54:49.598733_vgg16.h5
Epoch 2/10
131/131 [==============================] - 18s 135ms/step - loss: 0.3699 - accuracy: 0.8310 - val_loss: 0.2280 - val_accuracy: 0.9085

Epoch 00002: val_accuracy improved from 0.90402 to 0.90848, saving model to 2021-02-24 14:54:49.598733_vgg16.h5
Epoch 3/10
131/131 [==============================] - 18s 134ms/step - loss: 0.3111 - accuracy: 0.8661 - val_loss: 0.2207 - val_accuracy: 0.9040

Epoch 00003: val_accuracy did not improve from 0.90848
Epoch 4/10
131/131 [==============================] - 18s 136ms/step - loss: 0.3023 - accuracy: 0.8758 - val_loss: 0.2448 - val_accuracy: 0.8884

Epoch 00004: val_accuracy did not improve from 0.90848
Epoch 5/10
131/131 [==============================] - 18s 136ms/step - loss: 0.3227 -

In [ ]:
# load the best model
model = load_model(glob.glob('./*.h5')[0])
# build the the test generator
test_datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input)
test_generator=test_datagen.flow_from_directory(r"/content/Dataset/test",
                                                  target_size=(150,150),color_mode="rgb",
                                                  batch_size=1,class_mode="categorical")
# evaluate the model on a unseen data
scoreSeg = model.evaluate(test_generator)
print("Model Test accuracy: ",scoreSeg[1])
print("Model Test Loss: ",scoreSeg[0])

Found 450 images belonging to 2 classes.
450/450 [==============================] - 4s 8ms/step - loss: 0.2067 - accuracy: 0.9178
Model Test accuracy:  0.9177777767181396
Model Test Loss:  0.20666751265525818


In [ ]:
#load tensor board
%tensorboard --logdir logs/fit